In [2]:
import sys
import os

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 여러 동에 대한 반복문
dong_list = ['연희동', '후암동', '망원동', '연남동', '성수동']
for dong in dong_list:
    # 각 동에 해당하는 카페 리스트 파일 읽기
    cafe_list_path = f'/Users/hana/MJU/3-2/빅데이터프로그래밍/BDP_Proj/cafelist/cafelist_{dong}.csv'
    cafe_list = pd.read_csv(cafe_list_path)
    cafe_list = cafe_list.drop("Unnamed: 0", axis=1)
    cafe_list = cafe_list.drop("cafe_type", axis=1)
    
    for cafe in cafe_list["name"]:
        dong_name = dong.replace('동', '')
        if dong_name + '점' in cafe:
            cafe = cafe.replace(dong_name + '점', '')
        
        cafe = dong_name + " " + cafe
        cafe_keywords = cafe.split()
    
        # 파일 경로 설정
        folder_path = f"BDP-Project/{dong}/"
        file_path = os.path.join(folder_path, f"{cafe}_블로그_리뷰.csv")
        if not os.path.exists(file_path):

            #크롬 웹브라우저 실행
            chrome_options = webdriver.ChromeOptions()
            
            chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")
            driver = webdriver.Chrome()
            driver.get(
                "https://search.naver.com"
            )
            time.sleep(1)
            
            cafe_keywords = cafe.split()
    
            #검색창에 "검색어" 검색
            element = driver.find_element(By.CSS_SELECTOR, 'div.search_input_box > input#query')
            element.send_keys(dong+" "+cafe)
            element.submit()
            time.sleep(1)
        
            #VIEW 클릭
            driver.find_element(By.LINK_TEXT, "VIEW").click()
            
            #옵션 클릭
            driver.find_element(By.LINK_TEXT, "옵션").click()
            
            #블로그 클릭
            driver.find_element(By.LINK_TEXT, "블로그").click()
        
            url_list = []
            title_list = []
        
            # 스크롤을 밑으로 내려주는 함수
            def scroll_down(driver):
                driver.execute_script("window.scrollTo(0, 99999999)")
                time.sleep(1)
            
                # n : 스크롤할 횟수 설정
                n = 5 #스크롤 1번당 글 30개씩 화면에 출력
                i = 0
                while i < n:
                    scroll_down(driver) #스크롤다운
                    i = i+1
            
            
            #URL_raw 크롤링
            articles = "a.title_link"
            article_raw = driver.find_elements(By.CSS_SELECTOR, articles)
            for article in article_raw:
                title = article.text
        
                if all(keyword in title for keyword in cafe_keywords):
                    url = article.get_attribute('href')
                    url_list.append(url)
                    title_list.append(title)
        
                else:
                    continue
                    
            # 10개 미만으로 게시글을 가질 경우 검색 결과 상위 10개 추출
            if (len(url_list) < 10):
                url_list = []
                title_list = []

                article_raw = article_raw[:10]
                
                #URL_raw 크롤링
                for article in article_raw:
                    url = article.get_attribute('href')
                    url_list.append(url)
                
                    title = article.text
                    title_list.append(title)

            print("")
            print(cafe)
            print('url 갯수: ',len(url_list))
            print('title 갯수: ',len(title_list))
            
            df = {}
            df = pd.DataFrame({'url':url_list, 'title':title_list})
            
            # 폴더가 존재하지 않으면 생성
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
            file_name = f"{cafe}_블로그_리뷰.csv"
            
            # 파일 전체 경로 생성
            file_path = os.path.join(folder_path, file_name)
            
            # 데이터프레임을 CSV 파일로 저장
            df.to_csv(file_path, index=False)
            driver.close()
            
        else:
            print(f"File {file_path} already exists. Skipping…\n")
    
print(f"Crawling for {dong} completed.\n")

In [20]:
#포스트 띄우기
i = 0
url = df['url'][i]
driver.get(url)

In [21]:
iframes = driver.find_elements(By.CSS_SELECTOR, 'iframe')
for iframe in iframes:
    print(iframe.get_attribute('name'))

driver.switch_to.frame('mainFrame')
# try:
#     driver.switch_to.frame('mainFrame')
#     target_info = {} #개별 블로그 내용을 담을 딕셔너리 생성

#     #제목 크롤링
#     overlays = ".se-module.se-module-text.se-title-text"
#     tit = driver.find_element(By.CSS_SELECTOR, overlays)
#     title = tit.text


# #제목 크롤링
# overlays = ".se-module.se-module-text.se-title-text"
# tit = driver.find_element(By.CSS_SELECTOR, overlays)
# title = tit.text
# title


mainFrame


In [23]:
overlays = ".se-module.se-module-text.se-title-text"
tit = driver.find_element(By.CSS_SELECTOR, overlays)
title = tit.text
title

'[은평구카페] 필구커피 마쉬멜로우 시그니처메뉴'

In [26]:
overlays = ".se-main-container"
content= driver.find_element(By.CSS_SELECTOR, overlays)
cont = content.text
cont = cont.replace('\n', ' ')
cont

"필구커피 서울특별시 은평구 불광천길 442-29 1층 예약 안녕하세요 인프피 감성녀예요:) 필구커피 인스타에서 공지한대로 겨울 시즌메뉴로 11월 4일부터 선보이는 멜로우 라떼를 맛볼 겸, 은평구 필구커피에 다녀왔어요 필구커피 오픈시간 참고하세요 ~~ @filgucoffee 은평구에서 유명한 로컬 커피숍 필구커피는 주택을 개조하여 아기자기한 갬성의 카페예요 새절역에서 내리니 불광천 길이 시원스럽게 이어져 있구요^^ 평화롭게 산책하는 주민들이 많았어요 필구커피는 새절역에서 내려, 불광천 메인 길에서 조금 벗어나 주택가 쪽으로 5분정도 걸어 들어가면 나와요! 제가 도착한 시간은 오후 4시 경이었는데요~ 빵이없어요 ㅋㅋㅋ 텅 필구커피는 스콘맛집이라던데 안타깝게도 다음을 기약해봅니다 다음 방문 때에는 스콘 꼭 먹을꺼라구욧!! 필구커피 리뷰에서 가장 많이 보이는 메뉴인 멜로우 크림라떼도 안먹어봤는데, 하루에 두 잔을 마실 수는 없으니 이 또한 다음을 기약해봅니다 😓 💧 필구커피의 시그니처 메뉴인 멜로우 크림라떼는 벚꽃 🌸 시즌에는 핑크 마시멜로우가 토핑된다고 해요~! 로컬 커피숍으로서 지역주민들과의 소통을 중요하게 생각한다는 필구커피의 철학 ☕️ 지역민들 단골이 많은건지 테이크아웃 기다리는 사람들도 있더라구요 자 ~ 그럼 주문을 해볼까요 ? 저는 겨울 시즌메뉴 멜로우라떼와 레몬 마들렌을 주문했어요 ! 이렇게 귀여운 스티커 스탬프도 찍어주구요~ 유당불내증을 배려해서 소이라떼가 가능케 한 카페가 요즘 부쩍 눈에 띄는데요 ~ 필구커피도 두유로 비용 추가 없이 변경이 가능했어요! 밀크티는 귀여운 캐릭터가 그려진 병에 담아 주더라구요 !! 카페 내부는 최근 확장 공사를 해서 넓어졌다고 하는데 ~ 오늘이 첫 방문이어서 몰랐어요 😂 확장 공사를 했다고 하지만 아담한 느낌의 카페로 느껴졌답니다 !! LET'S GO EAT!! 필구커피 주문한 음료는 진동벨 없이 바리스타님이 직접 자리로 가져다 줘요! 필구커피의 메인 캐릭터 주황새와 함께 먹으러 가볼까요? 멜로우라떼 (5,000) 레몬

In [31]:
dict = {}
target_info = {}

target_info['title'] = title
target_info['content'] = cont

dict[0]=target_info
dict[0]

{'title': '[은평구카페] 필구커피 마쉬멜로우 시그니처메뉴',
 'content': "필구커피 서울특별시 은평구 불광천길 442-29 1층 예약 안녕하세요 인프피 감성녀예요:) 필구커피 인스타에서 공지한대로 겨울 시즌메뉴로 11월 4일부터 선보이는 멜로우 라떼를 맛볼 겸, 은평구 필구커피에 다녀왔어요 필구커피 오픈시간 참고하세요 ~~ @filgucoffee 은평구에서 유명한 로컬 커피숍 필구커피는 주택을 개조하여 아기자기한 갬성의 카페예요 새절역에서 내리니 불광천 길이 시원스럽게 이어져 있구요^^ 평화롭게 산책하는 주민들이 많았어요 필구커피는 새절역에서 내려, 불광천 메인 길에서 조금 벗어나 주택가 쪽으로 5분정도 걸어 들어가면 나와요! 제가 도착한 시간은 오후 4시 경이었는데요~ 빵이없어요 ㅋㅋㅋ 텅 필구커피는 스콘맛집이라던데 안타깝게도 다음을 기약해봅니다 다음 방문 때에는 스콘 꼭 먹을꺼라구욧!! 필구커피 리뷰에서 가장 많이 보이는 메뉴인 멜로우 크림라떼도 안먹어봤는데, 하루에 두 잔을 마실 수는 없으니 이 또한 다음을 기약해봅니다 😓 💧 필구커피의 시그니처 메뉴인 멜로우 크림라떼는 벚꽃 🌸 시즌에는 핑크 마시멜로우가 토핑된다고 해요~! 로컬 커피숍으로서 지역주민들과의 소통을 중요하게 생각한다는 필구커피의 철학 ☕️ 지역민들 단골이 많은건지 테이크아웃 기다리는 사람들도 있더라구요 자 ~ 그럼 주문을 해볼까요 ? 저는 겨울 시즌메뉴 멜로우라떼와 레몬 마들렌을 주문했어요 ! 이렇게 귀여운 스티커 스탬프도 찍어주구요~ 유당불내증을 배려해서 소이라떼가 가능케 한 카페가 요즘 부쩍 눈에 띄는데요 ~ 필구커피도 두유로 비용 추가 없이 변경이 가능했어요! 밀크티는 귀여운 캐릭터가 그려진 병에 담아 주더라구요 !! 카페 내부는 최근 확장 공사를 해서 넓어졌다고 하는데 ~ 오늘이 첫 방문이어서 몰랐어요 😂 확장 공사를 했다고 하지만 아담한 느낌의 카페로 느껴졌답니다 !! LET'S GO EAT!! 필구커피 주문한 음료는 진동벨 없이 바리스타님이 직접 자리로 가져다 

In [33]:
result_df = pd.DataFrame.from_dict(dict, orient='index')
result_df

,title,content
0,[은평구카페] 필구커피 마쉬멜로우 시그니처메뉴,필구커피 서울특별시 은평구 불광천길 442-29 1층 예약 안녕하세요 인프피 감성녀...


In [10]:
dict = {}

number = len(df)

for i in tqdm_notebook(range(0, number)):
    #글 띄우기
    url = df['url'][i]
    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)
    
    driver.get(url)

    try:
        driver.switch_to.frame('mainFrame')

        target_info = {}

        #제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"
        tit = driver.find_element(By.CSS_SELECTOR, overlays)
        title = tit.text

        #내용 크롤링
        overlays = ".se-main-container"
        content= driver.find_element(By.CSS_SELECTOR, overlays)
        cont = content.text
        cont = cont.replace('\n', ' ')

        #딕셔너리에 담음
        target_info['title'] = title
        target_info['content'] = cont

        #전체 딕셔너리에 담음
        dict[i]=target_info
        time.sleep(1)

        print(i, title)

        driver.close()

    except:
        driver.close()
        continue
    if i == len(df):
        result_df = pd.DataFrame.from_dict(dict, orient='index')

print('수집글갯수', len(dict))
print(dict)

  0%|          | 0/30 [00:00<?, ?it/s]

0 [은평구카페] 필구커피 마쉬멜로우 시그니처메뉴


1 필구커피, 은평구 카페, 연신내 카페


2 은평구 응암역 조용한 감성카페 필구커피


3 은평구 불광천 카페, 크림 라떼가 맛있는 필구커피


...

20 스물아홉의 일상) 서울숲카페 오브코하우스, 가로수길 정돈, 필구커피, 그리고 나이키덩크 코스트 지름


22 [리뷰]새절역 응암동 필구커피/은평구 핫플/불광천 카페(내돈내산)
23 [불광천 카페] 인스타 감성의 초소형 카페 필구 커피 FILGU COFFEE


24 필구커피 , 골목길 mood 그 자체 ...
25 필구커피 filgu coffee응암동 새절역 불광천카페


26 [새절역,응암동카페] 필구커피
27 [새절역카페]필구커피에서 멜로우라떼 마시고 반해버렸다.


28 [응암동 카페] 불광천 산책하다 가기 좋은 카페 ‘필구 커피’!_유쾌하신 사장님과 머쉬멜로우 커피 그리고 포인트 컬러가 매력적인 공간!
수집글갯수 28
{0: {'title': '[은평구카페] 필구커피 마쉬멜로우 시그니처메뉴', 'content': "필구커피 서울특별시 은평구 불광천길 442-29 1층 예약 안녕하세요 인프피 감성녀예요:) 필구커피 인스타에서 공지한대로 겨울 시즌메뉴로 11월 4일부터 선보이는 멜로우 라떼를 맛볼 겸, 은평구 필구커피에 다녀왔어요 필구커피 오픈시간 참고하세요 ~~ @filgucoffee 은평구에서 유명한 로컬 커피숍 필구커피는 주택을 개조하여 아기자기한 갬성의 카페예요 새절역에서 내리니 불광천 길이 시원스럽게 이어져 있구요^^ 평화롭게 산책하는 주민들이 많았어요 필구커피는 새절역에서 내려, 불광천 메인 길에서 조금 벗어나 주택가 쪽으로 5분정도 걸어 들어가면 나와요! 제가 도착한 시간은 오후 4시 경이었는데요~ 빵이없어요 ㅋㅋㅋ 텅 필구커피는 스콘맛집이라던데 안타깝게도 다음을 기약해봅니다 다음 방문 때에는 스콘 꼭 먹을꺼라구욧!! 필구커피 리뷰에서 가장 많이 보이는 메뉴인 멜로우 크림라떼도 안먹어봤는데, 하루에 두 잔을 마실 수는 없으니 이 또한 다음을 기약해봅니다 😓 💧 필구커피의 시그니처 메뉴인 멜로우 크림라떼는 벚꽃 🌸 시즌에는 핑크 마시멜로우가 토핑된다고 해요~! 로컬 커피숍으로서 지역주민들과의 소통을 중요하게 생각한다는 필구커피의 철학 ☕️ 지역민들 단골이 많은건지 테이크아웃 기다리는 사람들도 있더라구요 자 ~ 그럼 주문을 해볼까요 ? 저는 겨울 시즌메뉴 멜로우라떼와 레몬 마들렌을 주문했어요 ! 이렇게 귀여운 스티커 스탬프도 찍어주구요~ 유당불내증을 배려해서 소이라떼가 가능케 한 카페가 요즘 부쩍 눈에 띄는데요 ~ 필구커피도 두유로 비용 추가 없이 변경이 가능했어요! 밀크티는 귀여운 캐릭터가 그려진 병에 담아 주더라구요 !! 카페 내부는 최근 확장 공사를 해서 넓어졌다고 하는데 ~ 오늘이 첫 방문이어서 몰랐어요 😂

In [14]:
result = pd.DataFrame.from_dict(dict)
result.to_excel(query_txt+"_blog_content.xlsx", index=False)
time.sleep(3)